In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

from classifier import *
import features as fe 
import utils as ut
from sklearn.model_selection import train_test_split

## Feature Analysis
We look the following plots
- Time taken to convert image into integral immage vs size of the image.
- No of feature vectors vs size of the image
- Time taken to compute feature vectors vs size of the image. (sequential + parallel)
- Time to compute $i^{th}$ fixed vector for a fixed size of image.
- Time to compute $i^{th}$ feature vs size of the image.

### Time taken to convert image into integral immage vs size of the image.

### No of feature vectors vs size of the image

### Time taken to compute feature vectors vs size of the image.

### Time to compute $i^{th}$ fixed vector for a fixed size of image.

### Time to compute $i^{th}$ feature vs size of the image.

## Adaboost Analysis
We investigate the following plots
- Accuracy vs number of features used in the adaboost classifier.
- Positve Detection rate vs number of features used in the adaboost classifier.
- False Positive rate vs number of features used in the adaboost classifier.
- False Negative rate vs number of features used in the adaboost classifier
- Time taken to classify subwindows vs number of features used in adaboost classifier.

### Accuracy vs number of features used in the adaboost classifier.


### Positve Detection rate vs number of features used in the adaboost classifier.

### False Positive rate vs number of features used in the adaboost classifier.

### False Negative rate vs number of features used in the adaboost classifier

### Time taken to classify subwindows vs number of features used in adaboost classifier.

## Cascade Analysis
We investigate the following plots
- number of training samples for each layer.
- composition of training samples for each layer.
- overall best accuraciy results.
- Time taken to classify vs no of subwindows to be classified

### number of training samples for each layer.

### composition of training samples for each layer.

### Overall best accuraciy results

### Time taken to classify vs no of subwindows to be classified